In [2]:
import pandas as pd
import numpy as np

In [115]:
cases_deaths = pd.read_csv("https://data.chhs.ca.gov/dataset/f333528b-4d38-4814-bebb-12db1f10f535/resource/046cdd2b-31e5-4d34-9ed3-b48cdbc4be7a/download/covid19cases_test.csv")
vaccines = pd.read_csv("https://data.chhs.ca.gov/dataset/e283ee5a-cf18-4f20-a92c-ee94a2866ccd/resource/130d7ba2-b6eb-438d-a412-741bde207e1c/download/covid19vaccinesbycounty.csv")

In [116]:
cases_deaths["date"] = pd.to_datetime(cases_deaths["date"])

vaccines["date"] = pd.to_datetime(vaccines["administered_date"])
vaccines = vaccines.drop(["administered_date"], axis=1)

In [10]:
print(cases_deaths.columns)

Index(['date', 'area', 'area_type', 'population', 'cases', 'cumulative_cases',
       'deaths', 'cumulative_deaths', 'total_tests', 'cumulative_total_tests',
       'positive_tests', 'cumulative_positive_tests', 'reported_cases',
       'cumulative_reported_cases', 'reported_deaths',
       'cumulative_reported_deaths', 'reported_tests'],
      dtype='object')


In [120]:
vaccines.county = np.where(vaccines.county == "All CA Counties", "California", vaccines.county)

In [121]:
vaccines.county.unique()

array(['California', 'All CA and Non-CA Counties', 'Alpine', 'Amador',
       'Butte', 'Calaveras', 'Colusa', 'Del Norte', 'Glenn', 'Humboldt',
       'Imperial', 'Inyo', 'Kings', 'Lake', 'Lassen', 'Madera', 'Marin',
       'Mariposa', 'Mendocino', 'Merced', 'Modoc', 'Mono', 'Monterey',
       'Napa', 'Nevada', 'Plumas', 'San Benito', 'Santa Barbara',
       'Shasta', 'Sierra', 'Siskiyou', 'Stanislaus', 'Sutter', 'Tehama',
       'Trinity', 'Tulare', 'Tuolumne', 'Unknown', 'Yuba',
       'Outside California', 'Alameda', 'Contra Costa', 'El Dorado',
       'Fresno', 'Kern', 'Los Angeles', 'Orange', 'Placer', 'Riverside',
       'Sacramento', 'San Bernardino', 'San Diego', 'San Francisco',
       'San Joaquin', 'San Luis Obispo', 'San Mateo', 'Santa Clara',
       'Santa Cruz', 'Solano', 'Sonoma', 'Ventura', 'Yolo'], dtype=object)

In [122]:
combined = vaccines \
    .astype({"county": "string", "date": np.datetime64}) \
    .merge(
        cases_deaths[["date", "area", "population"]].astype({"area": "string"}),
        left_on=["county", "date"],
        right_on=["area","date"],
        how="inner"
    )
    
combined = combined[combined.county != "Unknown"]

In [124]:
all_counties_vaccines = combined[["county", "cumulative_at_least_one_dose", "date", "population", "cumulative_fully_vaccinated"]]
all_counties_vaccines \
    .sort_values(by=["county", "date"]) \
    .to_csv("../../dist/data/ca_vaccines.csv", index=False)

In [95]:
vaccines_data = combined[combined["county"] == "Santa Barbara"]
vaccines_data \
    .drop("county", axis=1) \
    .sort_values(by="date") \
    .to_csv("../../dist/data/vaccines.csv", index=False)